In [1]:
import os
from xml.dom import minidom
import pandas as pd
import numpy as np
directory = os.getcwd() 
import nltk
import gzip
import gensim 
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
# Extract text from inside

def preprocess(folder):
    no_of_paras = []
    current_dir = directory + "/"+folder
    base_cases = []
    entailed_fragments = []
    paragraphs = []
    list_of_files = sorted(os.listdir(current_dir))
    if folder == 'task2_train':
        x=1
    else:
        x = 2
    for i in range(len(list_of_files)-x):
        if i%100 == 0:
            print("Processing ",i)
        f = open(current_dir+'/'+list_of_files[i]+"/base_case.txt","r")
        g = open(current_dir+'/'+list_of_files[i]+"/entailed_fragment.txt","r")
        paragraph_filenames = sorted(os.listdir(current_dir+'/'+list_of_files[i]+"/paragraphs"))
        a = f.read()
        b = g.read()
        base_cases.append(a)
        entailed_fragments.append(b)
        f.close()
        g.close()
        
        arr = []
        no_of_paras.append(len(paragraph_filenames))

        for j in range(len(paragraph_filenames)):
            g = open(current_dir+'/'+list_of_files[i]+"/paragraphs/"+paragraph_filenames[j],"r")
            paragraphs.append(g.read())
            g.close()
    return base_cases,entailed_fragments,paragraphs,no_of_paras
    
train_base_cases,train_entailed_fragments, train_paragraphs,train_no_of_paras = preprocess('task2_train')
    
test_base_cases,test_entailed_fragments, test_paragraphs,test_no_of_paras = preprocess('task2_test')

Processing  0
Processing  100
Processing  0


In [ ]:
len(train_labels)

In [3]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
import re
from nltk.corpus import stopwords
lemma = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
dict_words = set(nltk.corpus.words.words())
def filt(case_docs):
    filtered_words =[]
    filtered_docs = [ '' for i in range(len(case_docs))]
    lemma_docs = [ '' for i in range(len(case_docs))]
    for i in range(len(case_docs)):
        case_words = re.split("(?:(?:[^a-zA-Z]+')|(?:'[^a-zA-Z]+))|(?:[^a-zA-Z']+)", case_docs[i])
        filtered_word_list = [word.lower() for word in case_words if (( len(word) >= 3 and word.isalpha() and word.lower() not in stop_words ))  ]
        filtered_words.append(filtered_word_list)
        '''or ( len(word) <= 3 and word.isdigit())'''
        for word in filtered_word_list:
            filtered_docs[i] = filtered_docs[i] + word + " "
            lemma_docs[i] = lemma_docs[i] + lemma.lemmatize(word) + " "
    return lemma_docs
train_base_cases = filt(train_base_cases)
train_entailed_fragments = filt(train_entailed_fragments)
train_paragraphs = filt(train_paragraphs)
test_entailed_fragments = filt(test_entailed_fragments)
test_paragraphs = filt(test_paragraphs)
test_base_cases = filt(test_base_cases)
#current_filtered_docs, current_filtered_words, current_lemma_docs = filt(current_case_docs,current_case_words)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
#corpus = test_entailed_fragments + test_paragraphs + test_base_cases
corpus =  test_entailed_fragments + test_paragraphs + test_base_cases
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
tf_idf = (X * X.T).A

In [5]:
cumm_para_list = []
tot = 0
for i in range(len(test_no_of_paras)):
    cumm_para_list.append(tot)
    tot += test_no_of_paras[i]

In [8]:
def calc(doc_no):
    lst = []
    doc_idx = doc_no-1
    for i in range(test_no_of_paras[doc_idx]):
        lst.append((tf_idf[doc_idx][44+cumm_para_list[doc_idx]+i],i+1))
    lst.sort(reverse = True)
    print("t2-"+str(doc_no),lst[0][1],"iitpTFItop10")
    for i in range(1,10):
        try:
            pred = lst[i][1]
            if pred >= 0.2:
                print("t2-"+str(doc_no),lst[i][1],"iitpTFItop10")
            else:
                break
        except:
            pass
for i in range(1, len(test_entailed_fragments)+1):
    calc(i)

t2-1 1 iitpTFItop10
t2-1 4 iitpTFItop10
t2-1 12 iitpTFItop10
t2-1 5 iitpTFItop10
t2-1 10 iitpTFItop10
t2-1 15 iitpTFItop10
t2-1 13 iitpTFItop10
t2-1 7 iitpTFItop10
t2-1 2 iitpTFItop10
t2-1 11 iitpTFItop10
t2-2 7 iitpTFItop10
t2-2 19 iitpTFItop10
t2-2 17 iitpTFItop10
t2-2 12 iitpTFItop10
t2-2 5 iitpTFItop10
t2-2 18 iitpTFItop10
t2-2 8 iitpTFItop10
t2-2 11 iitpTFItop10
t2-2 6 iitpTFItop10
t2-2 3 iitpTFItop10
t2-3 17 iitpTFItop10
t2-3 18 iitpTFItop10
t2-3 16 iitpTFItop10
t2-3 14 iitpTFItop10
t2-3 28 iitpTFItop10
t2-3 22 iitpTFItop10
t2-3 15 iitpTFItop10
t2-3 39 iitpTFItop10
t2-3 32 iitpTFItop10
t2-3 4 iitpTFItop10
t2-4 15 iitpTFItop10
t2-4 1 iitpTFItop10
t2-4 16 iitpTFItop10
t2-4 6 iitpTFItop10
t2-4 17 iitpTFItop10
t2-4 13 iitpTFItop10
t2-4 19 iitpTFItop10
t2-4 5 iitpTFItop10
t2-4 18 iitpTFItop10
t2-4 7 iitpTFItop10
t2-5 24 iitpTFItop10
t2-5 59 iitpTFItop10
t2-5 17 iitpTFItop10
t2-5 75 iitpTFItop10
t2-5 58 iitpTFItop10
t2-5 52 iitpTFItop10
t2-5 72 iitpTFItop10
t2-5 18 iitpTFItop10
t2-5 30